<a href="https://colab.research.google.com/github/paoguilar/POO/blob/main/finalproyect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [7]:
%%writefile app.py
import streamlit as st
import pandas as pd
from datetime import datetime, timedelta
import smtplib
import random
import os

# ----------📦 Configuración inicial del CSV----------
DATA_FILE = "cumples.csv"
MSG_FILE = "mensajes.csv"

def cargar_datos():
    if os.path.exists(DATA_FILE):
        return pd.read_csv(DATA_FILE)
    else:
        return pd.DataFrame(columns=["Nombre", "Email", "Fecha"])

def guardar_datos(df):
    df.to_csv(DATA_FILE, index=False)

def cargar_mensajes():
    if os.path.exists(MSG_FILE):
        return pd.read_csv(MSG_FILE)
    else:
        return pd.DataFrame(columns=["Mensaje"])

def guardar_mensajes(df):
    df.to_csv(MSG_FILE, index=False)

# ----------📤 Enviar correo----------
def enviar_correo(destinatario, asunto, mensaje, remitente, contraseña):
    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as servidor:
            servidor.starttls()
            servidor.login(remitente, contraseña)
            servidor.sendmail(remitente, destinatario, f"Subject: {asunto}\n\n{mensaje}")
    except Exception as e:
        st.error(f"❌ Error al enviar el correo: {e}")

# ----------📆 Funciones de cumpleaños----------
def dias_para_cumple(fecha_str):
    hoy = datetime.today()
    cumple = datetime.strptime(fecha_str, "%Y-%m-%d")
    cumple = cumple.replace(year=hoy.year)
    if cumple < hoy:
        cumple = cumple.replace(year=hoy.year + 1)
    return (cumple - hoy).days

def obtener_cumples_proximos(df):
    hoy = datetime.today()
    df["Dias"] = df["Fecha"].apply(lambda f: dias_para_cumple(f))
    return df.sort_values(by="Dias")

# ----------🎯 Interfaz Streamlit----------
st.set_page_config(page_title="Gestor de Cumpleaños 🎉", layout="centered")
st.title("🎈 Gestor de Cumpleaños")

df = cargar_datos()
df_msg = cargar_mensajes()

st.sidebar.header("📬 Configurar correo")
remitente = st.sidebar.text_input("Correo remitente (Gmail)", placeholder="tuemail@gmail.com")
contraseña = st.sidebar.text_input("Contraseña de aplicación", type="password")

# ----------🎂 Agregar cumpleaños----------
st.subheader("➕ Registrar nuevo cumpleaños")
with st.form("form_nuevo"):
    nombre = st.text_input("Nombre")
    email = st.text_input("Correo electrónico")
    fecha = st.date_input("Fecha de cumpleaños", min_value=datetime(1900, 1, 1))
    enviar_auto = st.checkbox("Enviar felicitación automáticamente")
    enviado = False
    enviado_hoy = False
    submit = st.form_submit_button("Guardar")

    if submit:
        nueva_fila = pd.DataFrame([{"Nombre": nombre, "Email": email, "Fecha": fecha.strftime("%Y-%m-%d")}])
        df = pd.concat([df, nueva_fila], ignore_index=True)
        guardar_datos(df)
        st.success(f"✅ Cumpleaños de {nombre} guardado correctamente")

# ----------📑 Ver cumpleaños próximos----------
st.subheader("📅 Cumpleaños próximos")
if not df.empty:
    df_mostrar = obtener_cumples_proximos(df)[["Nombre", "Fecha", "Email", "Dias"]]
    for _, row in df_mostrar.iterrows():
        dias = row["Dias"]
        mensaje = f"🎉 {row['Nombre']} - en {dias} día(s)" if dias > 0 else f"🎁 ¡Hoy es el cumpleaños de {row['Nombre']}!"
        st.write(mensaje)

        if dias == 0 and remitente and contraseña:
            if not df_msg.empty:
                mensaje_aleatorio = df_msg.sample(1)["Mensaje"].values[0]
            else:
                mensaje_aleatorio = f"¡Feliz cumpleaños, {row['Nombre']}! 🎂"

            enviar_correo(row["Email"], "🎉 ¡Feliz Cumpleaños!", mensaje_aleatorio, remitente, contraseña)
            st.success(f"📧 Correo enviado a {row['Nombre']}")
else:
    st.info("📭 No hay cumpleaños registrados.")

# ----------📝 Mensajes personalizados----------
st.subheader("💌 Configurar mensajes de cumpleaños")
nuevo_mensaje = st.text_area("Agregar nuevo mensaje", height=100)
if st.button("Guardar mensaje"):
    if nuevo_mensaje.strip():
        nueva_fila = pd.DataFrame([{"Mensaje": nuevo_mensaje.strip()}])
        df_msg = pd.concat([df_msg, nueva_fila], ignore_index=True)
        guardar_mensajes(df_msg)
        st.success("✅ Mensaje guardado")

if not df_msg.empty:
    st.write("📚 Mensajes guardados:")
    for i, row in df_msg.iterrows():
        st.write(f"{i+1}. {row['Mensaje']}")


Overwriting app.py


In [2]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 3s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.245.46.49
⠙⠹⠸⠼⠴⠦⠧your url is: https://proud-animals-crash.loca.lt
